In [1]:
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import patches
from matplotlib import pyplot as plt
from matplotlib import gridspec as gsp
from matplotlib.widgets import Slider, Button, RadioButtons
from ipywidgets import *

import numpy as np
import csv
import os
import glob
from functools import reduce
import SimpleITK as sitk
from scipy.spatial.distance import cdist


In [2]:
def get_data(locroot, mtype):
    truthlist = glob.glob(locroot+mtype+'/005/*/predictions/seg-*-tumor.nii')
    truthlist.sort()
    predlist  = glob.glob(locroot+mtype+'/005/*/predictions/pred-*-tumor-masked-int.nii')
    predlist.sort()
    assert len(truthlist) == len(predlist)
    print('Found', len(truthlist), 'segmentations and predictions...')
    return truthlist, predlist


In [3]:
# archs = ['unet', 'resnet', 'densenet']
# pocket = ['', '-pocket']
# c2Dt = ['', '-c2Dt']
archs = ['densenet']
pocket = ['-pocket']
c2Dt = ['']
modeltypes = []
for a in archs:
    for c in c2Dt:
        for p in pocket:
            modeltypes += [a+p+c]

locroot = '/rsrch1/ip/jacctor/smallnet/tumorhcc-ensemble/densenet-ensemble-512/'
# locroot = '/rsrch1/ip/jacctor/smallnet/tumorhcc-ensemble/e2e-512/'

In [4]:
def ShowImage(img):
    data = sitk.GetArrayFromImage(img)
    plt.imshow(data)
    plt.show()

In [5]:
def SliceViewer(data, data2=None):
    img = sitk.GetArrayFromImage(data)
    cmax = np.max(img)
    
    if data2 is not None:
        seg = sitk.GetArrayFromImage(data2)
        cmax = max([cmax, np.max(seg)])

    def f(Slice):
        if data2 is not None:
            fig = plt.figure(figsize=(8,4))
            ax  = fig.add_subplot(121)
        else:
            fig = plt.figure(figsize=(4,4))
            ax  = fig.add_subplot(111)
        img0 = img[...,Slice]
        ax.imshow(img0, cmap="hsv", vmin=0, vmax=cmax)
        if data2 is not None:
            ax = fig.add_subplot(122)
            seg0 = seg[...,Slice]
            ax.imshow(seg0, cmap="hsv", vmin=0, vmax=cmax)
    interact(f, Slice=widgets.IntSlider(min=0, max=img.shape[-1],step=1,value=0))

In [6]:
###
### true, pred are SITK::Image
### returns hdist_avg, hdist_max, dice, vol_true
###
def GetSITKStats(true, pred):
    
    # Calculate Hausdorff distances
    try:
        hausdorffFilter = sitk.HausdorffDistanceImageFilter()
#         hausdorffFilter.Execute(true, pred)
#         hdist_avg = hausdorffFilter.GetAverageHausdorffDistance()
#         hdist_max = hausdorffFilter.GetHausdorffDistance()
        hdist_avg = None
        hdist_max = None
    except:
        hdist_avg = None
        hdist_max = None
        
    
    # Calculate Dice scores
    try:
        diceFilter = sitk.LabelOverlapMeasuresImageFilter()
        diceFilter.Execute(true, pred)
        dice = diceFilter.GetDiceCoefficient()
    except:
        dice = None
    
    # Calculate true volume statistics
    try:
        stats = sitk.LabelShapeStatisticsImageFilter()
        stats.Execute(true)
        vol_true = stats.GetNumberOfPixels(1)
    except:
        vol_true = None
    
    results = (hdist_avg, hdist_max, dice, vol_true)
    return results

In [19]:
def closest_node(node, nodes):
    
    min_idx = cdist([node], nodes).argmin()
    return int(min_idx), nodes[min_idx]

def StatsSITK(truth, pred, iii):
    truth = sitk.ReadImage(truth, sitk.sitkUInt8, imageIO="NiftiImageIO")
    pred  = sitk.ReadImage(pred,  sitk.sitkUInt8, imageIO="NiftiImageIO")
    
    global_scores = GetSITKStats(truth, pred)
    individual_scores = {}

    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.Execute(truth)
    if stats.GetNumberOfLabels == 0:
        return global_scores, individual_scores
    
    individual_scores['true_vol']     = []
    individual_scores['closest-dice'] = []
    individual_scores['max-dice']     = []
    individual_scores['hdist_avg']    = []
    individual_scores['hdist_max']    = []
    
    
    connCompFilter      = sitk.ConnectedComponentImageFilter()
    conn_comp_true      = connCompFilter.Execute(truth)
    num_true_components = connCompFilter.GetObjectCount()
    true_centroids      = np.zeros((num_true_components+1, 3))
    print('\t Found', num_true_components, 'regions in true segmentation.')
    
    for idx in range(num_true_components+1):
        this_truth = sitk.BinaryThreshold(conn_comp_true,
                                  lowerThreshold=idx,
                                  upperThreshold=idx+1,
                                  insideValue=1,
                                  outsideValue=0)
        thisStatsFilter = sitk.LabelShapeStatisticsImageFilter()
        thisStatsFilter.Execute(this_truth)
#         print(idx, '\t', thisStatsFilter.GetNumberOfPixels(1), '\t', thisStatsFilter.GetCentroid(1))
        true_centroids[idx,:] = np.array(thisStatsFilter.GetCentroid(1))

        
    connCompFilter      = sitk.ConnectedComponentImageFilter()
    conn_comp_pred      = connCompFilter.Execute(pred)
    num_pred_components = connCompFilter.GetObjectCount()
    pred_centroids      = np.zeros((num_pred_components+1, 3))
    print('\t Found', num_pred_components, 'regions in predicted segmentation.')
    
    for idx in range(num_pred_components+1):
        this_pred = sitk.BinaryThreshold(conn_comp_pred,
                                  lowerThreshold=idx,
                                  upperThreshold=idx+1,
                                  insideValue=1,
                                  outsideValue=0)
        thisStatsFilter = sitk.LabelShapeStatisticsImageFilter()
        thisStatsFilter.Execute(this_pred)
#         print(idx, '\t', thisStatsFilter.GetNumberOfPixels(1), '\t', thisStatsFilter.GetCentroid(1))
        pred_centroids[idx,:] = np.array(thisStatsFilter.GetCentroid(1))

    dsc_matrix  = np.zeros((num_true_components+1, num_pred_components+1))
    closest_idx = np.zeros((num_true_components+1,), dtype=np.int16)
    maxdice_idx = np.zeros((num_true_components+1,), dtype=np.int16)
    for idx in range(num_true_components+1):
        this_truth = sitk.BinaryThreshold(conn_comp_true,
                                  lowerThreshold=idx,
                                  upperThreshold=idx+1,
                                  insideValue=1,
                                  outsideValue=0)
        
        pred_idx, _ = closest_node(true_centroids[idx,:], pred_centroids)
        closest_idx[idx] = pred_idx
        this_pred  = sitk.BinaryThreshold(conn_comp_pred,
                                  lowerThreshold=pred_idx,
                                  upperThreshold=pred_idx+1,
                                  insideValue=1,
                                  outsideValue=0)
        this_tumor_stats = GetSITKStats(this_truth, this_pred)
        individual_scores['hdist_avg']    += [this_tumor_stats[0]]
        individual_scores['hdist_max']    += [this_tumor_stats[1]]
        individual_scores['closest-dice'] += [this_tumor_stats[2]]
        individual_scores['true_vol']     += [this_tumor_stats[3]]
        
        for pred_idx in range(num_pred_components+1):
            this_pred  = sitk.BinaryThreshold(conn_comp_pred,
                                              lowerThreshold=pred_idx,
                                              upperThreshold=pred_idx+1,
                                              insideValue=1,
                                              outsideValue=0)

            this_tumor_stats = GetSITKStats(this_truth, this_pred)
            dsc_matrix[idx, pred_idx] = this_tumor_stats[2]
    
    maxdice_idx = np.argmax(dsc_matrix, axis=-1)
    maxdice     = np.max(dsc_matrix, axis=-1)
    closedice   = np.array( [dsc_matrix[idx, closest_idx[idx]] for idx in range(dsc_matrix.shape[0])])

    print('\t Global dice: {:.4f}'.format(global_scores[2]))
    for idx in range(num_true_components+1):
        print('\t', idx, '\tDice (closest): {:.4f}'.format(closedice[idx]), '\tDice (max): {:.4f}'.format(maxdice[idx]))
    
    gs = gsp.GridSpec(1,3, width_ratios=[num_pred_components+1,1,1])
    fig = plt.figure()
    ax1 = plt.subplot(gs[0])
    ax1.imshow(dsc_matrix, cmap='gray', aspect='equal')
    ax1.title.set_text('matrix')
    for idx in range(num_true_components+1):
        pred_idx = closest_idx[idx]
        max_idx  = maxdice_idx[idx]
        rect = patches.Rectangle((max_idx-0.5,  idx-0.5), 1, 1, edgecolor='g', facecolor="none", linewidth=3)
        ax1.add_patch(rect)
        rect = patches.Rectangle((pred_idx-0.5, idx-0.5), 1, 1, edgecolor='r', facecolor="none", linewidth=3)
        ax1.add_patch(rect)
    
    ax2 = plt.subplot(gs[1])
    ax2.imshow(maxdice[:, np.newaxis],   cmap='gray')
    ax2.title.set_text('max')
    ax2.set_aspect('equal')
    ax2.xaxis.set_visible(False)
    
    ax3 = plt.subplot(gs[2])
    ax3.imshow(closedice[:, np.newaxis], cmap='gray')
    ax3.title.set_text('closest')
    ax3.set_aspect('equal')
    ax3.xaxis.set_visible(False)
    
    plt.savefig('dice-matrices/dice-matrix-'+str(iii)+'.png')
    plt.close()
    plt.clf()
    
    for idx in range(num_true_components+1):
        individual_scores['max-dice'] += [maxdice[idx]]
    
    # filter out the false positives, and recompute global scores
    closest_seg = sitk.Image( *truth.GetSize(), sitk.sitkUInt8)
    for idx in range(1,num_true_components+1):
        this_close_idx = int(closest_idx[idx])
        if this_close_idx != 0:
            this_close_pred = sitk.BinaryThreshold(conn_comp_pred,
                                                lowerThreshold=this_close_idx,
                                                upperThreshold=this_close_idx+1,
                                                insideValue=1,
                                                outsideValue=0)
            closest_seg = sitk.Or(closest_seg, this_close_pred)
    filtered_closest_stats = GetSITKStats(truth, closest_seg)
    print('\t Filtered dice (closest): {:.4f}'.format(filtered_closest_stats[2]))

        
    maxreg_seg  = sitk.Image( *truth.GetSize(), sitk.sitkUInt8)
    for idx in range(1,num_true_components+1):
        this_max_idx = int(maxdice_idx[idx])
        if this_max_idx == 0:
            this_max_idx = int(closest_idx[idx])
        if this_max_idx != 0:
            this_max_pred = sitk.BinaryThreshold(conn_comp_pred,
                                                lowerThreshold=this_max_idx,
                                                upperThreshold=this_max_idx+1,
                                                insideValue=1,
                                                outsideValue=0)
            maxreg_seg = sitk.Or(maxreg_seg, this_max_pred)
    filtered_maxreg_stats  = GetSITKStats(truth, maxreg_seg)
    print('\t Filtered dice     (max): {:.4f}'.format(filtered_maxreg_stats[2]))
    
    filtered_scores = [None]*2
    filtered_scores[0] = filtered_closest_stats[2]
    filtered_scores[1] = filtered_maxreg_stats[2]
    
    return global_scores, individual_scores, filtered_scores


In [34]:
mtype = modeltypes[0]
print('Model type', mtype)
truthlist, predlist = get_data(locroot, mtype)

# truthlist = truthlist[:25]
# predlist  = predlist[:25]

Model type densenet-pocket
Found 131 segmentations and predictions...


In [35]:
print('Calculating Hausdorff distances...')
hdist_avg = [None]*len(truthlist)
hdist_max = [None]*len(truthlist)
label_sizes = [None]*len(truthlist)
total_sizes = [None]*len(truthlist)
dice_scores = [None]*len(truthlist)
dice_max_scores = [None]*len(truthlist)
dice_closest_scores = [None]*len(truthlist)
individual_stats = [None]*len(truthlist)
for idx, (tr, pr) in enumerate(zip(truthlist, predlist)):
    print('Image', idx)
    global_scores, individual_scores, filtered_scores = StatsSITK(tr, pr, idx)
    hdist_avg[idx] = global_scores[0]
    hdist_max[idx] = global_scores[1]
    dice_scores[idx] = global_scores[2]
    total_sizes[idx] = global_scores[3]
    label_sizes[idx] = individual_scores['true_vol']
    individual_stats[idx] = individual_scores
    dice_closest_scores[idx] = filtered_scores[0]
    dice_max_scores[idx] = filtered_scores[1]


    
    

Calculating Hausdorff distances...
Image 0
	 Found 11 regions in true segmentation.
	 Found 10 regions in predicted segmentation.
	 Global dice: 0.3991
	 0 	Dice (closest): 0.9999 	Dice (max): 0.9999
	 1 	Dice (closest): 0.5699 	Dice (max): 0.5699
	 2 	Dice (closest): 0.6416 	Dice (max): 0.6416
	 3 	Dice (closest): 0.3314 	Dice (max): 0.3314
	 4 	Dice (closest): 0.1847 	Dice (max): 0.4715
	 5 	Dice (closest): 0.3483 	Dice (max): 0.3483
	 6 	Dice (closest): 0.0000 	Dice (max): 0.2875
	 7 	Dice (closest): 0.0000 	Dice (max): 0.0001
	 8 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 9 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 10 	Dice (closest): 0.3990 	Dice (max): 0.3990
	 11 	Dice (closest): 0.6142 	Dice (max): 0.6142
	 Filtered dice (closest): 0.0003
	 Filtered dice     (max): 0.4220
Image 1
	 Found 12 regions in true segmentation.
	 Found 15 regions in predicted segmentation.
	 Global dice: 0.5824
	 0 	Dice (closest): 0.9999 	Dice (max): 0.9999
	 1 	Dice (closest): 0.7192 	Dice (ma

	 Filtered dice (closest): 0.5684
	 Filtered dice     (max): 0.5684
Image 16
	 Found 1 regions in true segmentation.
	 Found 26 regions in predicted segmentation.
	 Global dice: 0.6253
	 0 	Dice (closest): 0.9997 	Dice (max): 0.9997
	 1 	Dice (closest): 0.6630 	Dice (max): 0.6664
	 Filtered dice (closest): 0.6630
	 Filtered dice     (max): 0.6664
Image 17
	 Found 1 regions in true segmentation.
	 Found 10 regions in predicted segmentation.
	 Global dice: 0.2080
	 0 	Dice (closest): 1.0000 	Dice (max): 1.0000
	 1 	Dice (closest): 0.4544 	Dice (max): 0.4544
	 Filtered dice (closest): 0.4544
	 Filtered dice     (max): 0.4544
Image 18
	 Found 1 regions in true segmentation.
	 Found 7 regions in predicted segmentation.
	 Global dice: 0.3020
	 0 	Dice (closest): 1.0000 	Dice (max): 1.0000
	 1 	Dice (closest): 0.4929 	Dice (max): 0.4929
	 Filtered dice (closest): 0.4929
	 Filtered dice     (max): 0.4929
Image 19
	 Found 9 regions in true segmentation.
	 Found 5 regions in predicted segmentati

	 Found 8 regions in predicted segmentation.
	 Global dice: 0.5943
	 0 	Dice (closest): 0.9999 	Dice (max): 0.9999
	 1 	Dice (closest): 0.0866 	Dice (max): 0.0866
	 2 	Dice (closest): 0.7098 	Dice (max): 0.7098
	 3 	Dice (closest): 0.6479 	Dice (max): 0.6479
	 4 	Dice (closest): 0.7794 	Dice (max): 0.7794
	 5 	Dice (closest): 0.1673 	Dice (max): 0.1673
	 6 	Dice (closest): 0.1341 	Dice (max): 0.1656
	 7 	Dice (closest): 0.5047 	Dice (max): 0.5055
	 8 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 Filtered dice (closest): 0.5943
	 Filtered dice     (max): 0.5943
Image 32
	 Found 0 regions in true segmentation.
	 Found 0 regions in predicted segmentation.
	 Global dice: inf
	 0 	Dice (closest): 1.0000 	Dice (max): 1.0000
	 Filtered dice (closest): inf
	 Filtered dice     (max): inf
Image 33
	 Found 15 regions in true segmentation.
	 Found 21 regions in predicted segmentation.
	 Global dice: 0.7111
	 0 	Dice (closest): 0.9973 	Dice (max): 0.9973
	 1 	Dice (closest): 0.6887 	Dice (max): 0.6

	 Filtered dice (closest): 0.0025
	 Filtered dice     (max): 0.0025
Image 47
	 Found 0 regions in true segmentation.
	 Found 1 regions in predicted segmentation.
	 Global dice: 0.0000
	 0 	Dice (closest): 1.0000 	Dice (max): 1.0000
	 Filtered dice (closest): inf
	 Filtered dice     (max): inf
Image 48
	 Found 4 regions in true segmentation.
	 Found 2 regions in predicted segmentation.
	 Global dice: 0.8027
	 0 	Dice (closest): 0.9997 	Dice (max): 0.9997
	 1 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 2 	Dice (closest): 0.8150 	Dice (max): 0.8150
	 3 	Dice (closest): 0.8084 	Dice (max): 0.8084
	 4 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 Filtered dice (closest): 0.0011
	 Filtered dice     (max): 0.0011
Image 49
	 Found 22 regions in true segmentation.
	 Found 4 regions in predicted segmentation.
	 Global dice: 0.5951
	 0 	Dice (closest): 1.0000 	Dice (max): 1.0000
	 1 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 2 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 3 	Dice (closest): 

	 Global dice: 0.0000
	 0 	Dice (closest): 1.0000 	Dice (max): 1.0000
	 1 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 Filtered dice (closest): 0.0000
	 Filtered dice     (max): 0.0000
Image 68
	 Found 1 regions in true segmentation.
	 Found 37 regions in predicted segmentation.
	 Global dice: 0.0549
	 0 	Dice (closest): 0.9990 	Dice (max): 0.9990
	 1 	Dice (closest): 0.8508 	Dice (max): 0.8516
	 Filtered dice (closest): 0.8508
	 Filtered dice     (max): 0.8516
Image 69
	 Found 1 regions in true segmentation.
	 Found 3 regions in predicted segmentation.
	 Global dice: 0.7915
	 0 	Dice (closest): 1.0000 	Dice (max): 1.0000
	 1 	Dice (closest): 0.8168 	Dice (max): 0.8168
	 Filtered dice (closest): 0.8168
	 Filtered dice     (max): 0.8168
Image 70
	 Found 3 regions in true segmentation.
	 Found 43 regions in predicted segmentation.
	 Global dice: 0.8260
	 0 	Dice (closest): 0.9997 	Dice (max): 0.9997
	 1 	Dice (closest): 0.7076 	Dice (max): 0.7111
	 2 	Dice (closest): 0.8734 	Dice (max):

	 Filtered dice (closest): 0.0027
	 Filtered dice     (max): 0.0027
Image 84
	 Found 10 regions in true segmentation.
	 Found 10 regions in predicted segmentation.
	 Global dice: 0.5727
	 0 	Dice (closest): 0.9999 	Dice (max): 0.9999
	 1 	Dice (closest): 0.6686 	Dice (max): 0.7017
	 2 	Dice (closest): 0.8128 	Dice (max): 0.8128
	 3 	Dice (closest): 0.0785 	Dice (max): 0.0785
	 4 	Dice (closest): 0.6610 	Dice (max): 0.6610
	 5 	Dice (closest): 0.7342 	Dice (max): 0.7342
	 6 	Dice (closest): 0.0000 	Dice (max): 0.3128
	 7 	Dice (closest): 0.5608 	Dice (max): 0.5608
	 8 	Dice (closest): 0.3881 	Dice (max): 0.5076
	 9 	Dice (closest): 0.4384 	Dice (max): 0.4384
	 10 	Dice (closest): 0.4773 	Dice (max): 0.4773
	 Filtered dice (closest): 0.6000
	 Filtered dice     (max): 0.6138
Image 85
	 Found 2 regions in true segmentation.
	 Found 11 regions in predicted segmentation.
	 Global dice: 0.5755
	 0 	Dice (closest): 0.9995 	Dice (max): 0.9995
	 1 	Dice (closest): 0.5475 	Dice (max): 0.5475
	 2 

	 Filtered dice (closest): 0.4814
	 Filtered dice     (max): 0.3637
Image 100
	 Found 6 regions in true segmentation.
	 Found 48 regions in predicted segmentation.
	 Global dice: 0.0000
	 0 	Dice (closest): 0.9999 	Dice (max): 0.9999
	 1 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 2 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 3 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 4 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 5 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 6 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 Filtered dice (closest): 0.0000
	 Filtered dice     (max): 0.0000
Image 101
	 Found 29 regions in true segmentation.
	 Found 11 regions in predicted segmentation.
	 Global dice: 0.3335
	 0 	Dice (closest): 0.9999 	Dice (max): 0.9999
	 1 	Dice (closest): 0.0000 	Dice (max): 0.0001
	 2 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 3 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 4 	Dice (closest): 0.3184 	Dice (max): 0.5302
	 5 	Dice (closest): 0.3283 	Dice (max): 0.5450
	 6

	 Filtered dice (closest): 0.2666
	 Filtered dice     (max): 0.3234
Image 109
	 Found 30 regions in true segmentation.
	 Found 18 regions in predicted segmentation.
	 Global dice: 0.0548
	 0 	Dice (closest): 0.9998 	Dice (max): 0.9998
	 1 	Dice (closest): 0.1807 	Dice (max): 0.1807
	 2 	Dice (closest): 0.2115 	Dice (max): 0.2115
	 3 	Dice (closest): 0.1661 	Dice (max): 0.3140
	 4 	Dice (closest): 0.0000 	Dice (max): 0.0270
	 5 	Dice (closest): 0.0000 	Dice (max): 0.0012
	 6 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 7 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 8 	Dice (closest): 0.0926 	Dice (max): 0.0966
	 9 	Dice (closest): 0.2108 	Dice (max): 0.2149
	 10 	Dice (closest): 0.3240 	Dice (max): 0.3240
	 11 	Dice (closest): 0.0000 	Dice (max): 0.0001
	 12 	Dice (closest): 0.0000 	Dice (max): 0.0001
	 13 	Dice (closest): 0.1464 	Dice (max): 0.1464
	 14 	Dice (closest): 0.1322 	Dice (max): 0.1322
	 15 	Dice (closest): 0.0000 	Dice (max): 0.1355
	 16 	Dice (closest): 0.0000 	Dice (max

	 Filtered dice (closest): 0.2753
	 Filtered dice     (max): 0.2879
Image 118
	 Found 11 regions in true segmentation.
	 Found 38 regions in predicted segmentation.
	 Global dice: 0.1883
	 0 	Dice (closest): 0.9983 	Dice (max): 0.9983
	 1 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 2 	Dice (closest): 0.0068 	Dice (max): 0.0524
	 3 	Dice (closest): 0.0068 	Dice (max): 0.0524
	 4 	Dice (closest): 0.0000 	Dice (max): 0.0780
	 5 	Dice (closest): 0.0000 	Dice (max): 0.0632
	 6 	Dice (closest): 0.0163 	Dice (max): 0.0163
	 7 	Dice (closest): 0.0173 	Dice (max): 0.0173
	 8 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 9 	Dice (closest): 0.0000 	Dice (max): 0.5656
	 10 	Dice (closest): 0.6260 	Dice (max): 0.6260
	 11 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 Filtered dice (closest): 0.0068
	 Filtered dice     (max): 0.0069
Image 119
	 Found 0 regions in true segmentation.
	 Found 2 regions in predicted segmentation.
	 Global dice: 0.0000
	 0 	Dice (closest): 1.0000 	Dice (max): 1.0000
	 

	 Filtered dice (closest): 0.0061
	 Filtered dice     (max): 0.0063


In [56]:
import time

mtype = modeltypes[0]
print('Model type', mtype)
my_truthlist, my_predlist = get_data(locroot, mtype)

my_truthlist = my_truthlist[:1]
my_predlist  = my_predlist[:1]

start_time = time.time()

print('Calculating Hausdorff distances...')
my_hdist_avg = [None]*len(my_truthlist)
my_hdist_max = [None]*len(my_truthlist)
my_label_sizes = [None]*len(my_truthlist)
my_total_sizes = [None]*len(my_truthlist)
my_dice_scores = [None]*len(my_truthlist)
my_dice_max_scores = [None]*len(my_truthlist)
my_dice_closest_scores = [None]*len(my_truthlist)
my_individual_stats = [None]*len(my_truthlist)
for idx, (tr, pr) in enumerate(zip(my_truthlist, my_predlist)):
    print('Image', idx)
    my_global_scores, my_individual_scores, my_filtered_scores = StatsSITK(tr, pr, idx)
    my_hdist_avg[idx] = my_global_scores[0]
    my_hdist_max[idx] = my_global_scores[1]
    my_dice_scores[idx] = my_global_scores[2]
    my_total_sizes[idx] = my_global_scores[3]
    my_label_sizes[idx] = my_individual_scores['true_vol']
    my_individual_stats[idx] = my_individual_scores
    my_dice_closest_scores[idx] = my_filtered_scores[0]
    my_dice_max_scores[idx] = my_filtered_scores[1]

end_time = time.time()
elapsed = end_time - start_time
print('\nTime elapsed:\t', elapsed)



Model type densenet-pocket
Found 131 segmentations and predictions...
Calculating Hausdorff distances...
Image 0
	 Found 11 regions in true segmentation.
	 Found 10 regions in predicted segmentation.
	 Global dice: 0.3991
	 0 	Dice (closest): 0.9999 	Dice (max): 0.9999
	 1 	Dice (closest): 0.5699 	Dice (max): 0.5699
	 2 	Dice (closest): 0.6416 	Dice (max): 0.6416
	 3 	Dice (closest): 0.3314 	Dice (max): 0.3314
	 4 	Dice (closest): 0.1847 	Dice (max): 0.4715
	 5 	Dice (closest): 0.3483 	Dice (max): 0.3483
	 6 	Dice (closest): 0.0000 	Dice (max): 0.2875
	 7 	Dice (closest): 0.0000 	Dice (max): 0.0001
	 8 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 9 	Dice (closest): 0.0000 	Dice (max): 0.0000
	 10 	Dice (closest): 0.3990 	Dice (max): 0.3990
	 11 	Dice (closest): 0.6142 	Dice (max): 0.6142
	 Filtered dice (closest): 0.0003
	 Filtered dice     (max): 0.4220

Time elapsed:	 26.635648250579834


In [36]:
plt.figure(figsize=(12,4))
plt.scatter(list(range(len(dice_scores))), dice_scores, c='black')
plt.scatter(list(range(len(dice_closest_scores))), dice_closest_scores, c='r')
plt.scatter(list(range(len(dice_max_scores))), dice_max_scores, c='g')
plt.savefig('global-tumor-dice-results.png')
plt.close()
plt.clf()

In [55]:
plt.figure(figsize=(4,4))
plt.hist(each_tumor_volume, bins=100, range=[0,20000])
plt.savefig('tumor-volume-hist.png')
plt.close()
plt.clf()

In [42]:
dicediff_closest = np.array(dice_closest_scores) - np.array(dice_scores)
dicediff_max     = np.array(dice_max_scores)     - np.array(dice_scores)

dicediff_closest = dicediff_closest[np.isfinite(dicediff_closest)]
dicediff_max     = dicediff_max[np.isfinite(dicediff_max)]

plt.figure(figsize=(8,4))
ax = plt.subplot(121)
plt.hist(dicediff_closest, bins=50)
ax.set_title('Closest Centroid')
ax = plt.subplot(122)
plt.hist(dicediff_max, bins=50)
ax.set_title('Max Dice')
plt.savefig('global-tumor-dice-difference-filtering.png')
plt.close()
plt.clf()

/opt/apps/miniconda/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in subtract
  """Entry point for launching an IPython kernel.
/opt/apps/miniconda/maskrcnn/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in subtract
  


In [37]:
    
# Individual Dice Statistics
each_tumor_dice      = []
each_tumor_dice_max  = []
each_tumor_volume    = []
for idx in range(len(truthlist)):
    if individual_stats[idx] is not None:
        each_tumor_dice      += individual_stats[idx]['closest-dice']
        each_tumor_dice_max  += individual_stats[idx]['max-dice']
        each_tumor_volume    += individual_stats[idx]['true_vol']
print(len(each_tumor_dice))
print(len(each_tumor_volume))
print('Number of zero dice scores:', len([v for v in each_tumor_dice     if v <= 0.01]))
print('Number of zero dice scores:', len([v for v in each_tumor_dice_max if v <= 0.01]))


plt.figure(figsize=(8,4))
ax = plt.subplot(121)
plt.scatter( each_tumor_volume, each_tumor_dice)
ax.set_title('Volume vs. Dice of Closest Centroid')
plt.xlim([0,7.e4])
ax = plt.subplot(122)
plt.scatter( each_tumor_volume, each_tumor_dice_max)
ax.set_title('Volume vs. Dice of Max Predicted Region')
plt.xlim(0,7.e4)
plt.savefig('individual-tumor-dice-results.png')
plt.close()
plt.clf()


1168
1168
Number of zero dice scores: 548
Number of zero dice scores: 451


In [51]:
np_each_tumor_dice = np.array( each_tumor_dice )
np_each_tumor_dice_max = np.array( each_tumor_dice_max )
np_dice_scores = np.array( dice_scores )
np_dice_scores_closest = np.array( dice_closest_scores )
np_dice_scores_max     = np.array( dice_max_scores )

def filter_inf(x):
    x[np.isinf(x)] = 0.0
    return x

np_dice_scores = filter_inf(np_dice_scores)
np_dice_scores_closest = filter_inf(np_dice_scores_closest)
np_dice_scores_max = filter_inf(np_dice_scores_max)

print('Global          \t', np.mean( np_dice_scores), np.std(np_dice_scores))
print('Global - closest\t', np.mean( np_dice_scores_closest), np.std(np_dice_scores_closest))
print('Global -     max\t', np.mean( np_dice_scores_max), np.std(np_dice_scores_max))
print('Tumor  - closest\t', np.mean( np_each_tumor_dice), np.std( np_each_tumor_dice))
print('Tumor  -     max\t', np.mean( np_each_tumor_dice_max), np.std( np_each_tumor_dice_max))

Global          	 0.3126072408876341 0.29612112571082655
Global - closest	 0.2975849039029823 0.30555854339674854
Global -     max	 0.30990257631592977 0.30200981273444893
Tumor  - closest	 0.283695892428425 0.3594167612673872
Tumor  -     max	 0.3151079232921547 0.3534298500934147


In [ ]:
# filtering for images without tumors        
hdist_avg = [h for h in hdist_avg if h is not None]
hdist_max = [h for h in hdist_max if h is not None]
label_sizes = [l for l in label_sizes if l is not None]
total_sizes = [t for t in total_sizes if t is not None]

# calculating statistics
mean_hdist = sum(hdist_avg) / len(hdist_avg)
max_hdist  = sum(hdist_max) / len(hdist_max)
print('')
print('Hausdorff distances: \t mean:', mean_hdist, '\t max:', max_hdist)
print('Label sizes: \t', label_sizes)

In [ ]:
plt.figure(figsize=(12,4))
ax = plt.subplot(1,3,1)
plt.scatter( total_sizes, hdist_avg)
ax.set_title('Volume vs. Mean Hausdorff')
ax = plt.subplot(1,3,2)
plt.scatter( total_sizes, hdist_max)
ax.set_title('Volume vs. Max Hausdorff')
ax = plt.subplot(1,3,3)
plt.scatter( hdist_avg, hdist_max)
ax.set_title('Mean Hausdorff vs. Max Hausdorff')
plt.show()


Cells below are to collate results saved to the output.txt files

In [ ]:
def collate_model_comparisons(mtype, locroot='', separate_folds=False):
    print('\ncollating stats for base model', mtype)
    
    textfilelist = glob.glob(locroot+mtype+'/output-005-*.txt')
    print('number of files found:\t', len(textfilelist), '\n')
    
    if separate_folds:
        textfilelist_brokenstrings = [f.split('\\') for f in textfilelist]
    else:
        textfilelist_brokenstrings = [f.split('/') for f in textfilelist]
    textfilelist_dict = {}
    for idx, t in enumerate(textfilelist_brokenstrings):
        try:
            textfilelist_dict[t[0]] += [textfilelist[idx]]
        except:
            textfilelist_dict[t[0]] = [textfilelist[idx]]

    for model in textfilelist_dict:
        print(model)
        scores_dict = gather_test_scores(textfilelist_dict[model])
        plt.figure(figsize=(12,4))
        for idx, metric in enumerate(scores_dict):
            print(metric, np.mean(np.array(scores_dict[metric])), np.std(np.array(scores_dict[metric])))
            ax = plt.subplot(1,3,idx+1)
            plt.hist(scores_dict[metric], range=[0,1], bins=25)
            ax.set_title(metric)
        plt.show()
        print()
    return scores_dict

In [ ]:
archs = ['densenet']
pocket = ['', '-pocket']
modeltypes = []
for a in archs:
    for p in pocket:
        modeltypes += [a+p]

In [ ]:
def initialize_dict(textfile, n_expected=3):
    with open(textfile, 'r') as f:
        linelist = f.readlines()
        testscores = [line for line in linelist if line[0]=='+']
        individual_scores = dict()
        try:
            lines = testscores[:n_expected]
            for l in lines:
                wordlist = l.split()
                name = reduce(lambda x, y: x+' '+y, wordlist[1:-2])
                individual_scores[name] = []
        except:
            print('cannot process file to initialize dict')
        return individual_scores

def process_textfile(textfilelist, score_dict):
    for textfile in textfilelist:
        with open(textfile, 'r') as f:
            linelist = f.readlines()
            testscores = [line for line in linelist if line[0]=='+']
            try:
                for line in testscores:
                    splitline = line.split()
                    name = reduce(lambda x, y: x+' '+y, splitline[1:-2])
                    try:
                        score_dict[name] += [float(splitline[-1])]
                    except:
#                         print('failed to add', name)
                        continue
#                 metrics = [m[1:-2] for m in testscores[0][1:].split()]
#                 assert list(score_dict.keys()) == metrics
#                 for test_img in testscores[2:]:
#                     scores = [float(s[:-1]) for s in test_img[1:].split()]
#                     for i in range(len(metrics)):
#                         score_dict[metrics[i]] += [scores[i]] 
            except:
                print('cannot process file:', textfile)
    return score_dict

def gather_test_scores(textfilelist):
    score_dict = initialize_dict(textfilelist[0])
    return process_textfile(textfilelist, score_dict)

In [ ]:
# modeltypes = ['unet', 'resnet', 'densenet']
results_dict = {}
for mtype in modeltypes:
    results_dict[mtype] = collate_model_comparisons(mtype, separate_folds=False, locroot='../densenet-ensemble-512/')
    

In [ ]:
plt.figure(figsize=(12,12))
idx = 0
for model in results_dict:
    idx += 1
    metric = list(results_dict[model].keys())[2]
    this_model_results = results_dict[model][metric]
    ax = plt.subplot(3,4,idx)
    plt.boxplot(this_model_results)
    plt.ylim([0,1])
    ax.set_title(model)
plt.savefig('dsc-comp-boxplot.png')
plt.show()

    

In [ ]:
import scipy
from scipy.stats import wilcoxon, mannwhitneyu

model_names = [('unet', 'unet-pocket'),
               ('resnet', 'resnet-pocket'),
               ('densenet', 'densenet-pocket'),
               ('unet-c2Dt', 'unet-pocket-c2Dt'),
               ('resnet-c2Dt', 'resnet-pocket-c2Dt'),
               ('densenet-c2Dt', 'densenet-pocket-c2Dt')]
for model in model_names:
    print('Testing', model[0], 'vs.', model[1])
    metric = list(results_dict[model[0]].keys())[2]
    this_model_results_base = results_dict[model[0]][metric]
    this_model_results_pocket = results_dict[model[1]][metric]
    diff = [r1 - r2 for r1, r2 in zip(this_model_results_base, this_model_results_pocket)]
    wiltest = wilcoxon(diff, correction=True)
    mwtest  = mannwhitneyu(this_model_results_base, this_model_results_pocket)
    print(wiltest2, len(diff))
    print(mwtest,   len(diff))
    print('\n')

In [ ]:
this_model_results_base